In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from yaml import safe_load, YAMLError
from src.snapshots import snapshots_assembly
from src.dim_reduction import SVD, AutoEncoderTrain, AutoEncoder

In [5]:
with open("parameters.yaml", "r") as stream:
    try:
        params = safe_load(stream)
    except YAMLError as exc:
        print(exc)

In [6]:
params

{'snapshots': {'file_type_str': 'h5_fenics',
  'folder': 'data',
  'file_name_contains': ['concentration'],
  'dataset': None}}

In [7]:
filenames, snapshots = snapshots_assembly(params["snapshots"])

2024-01-12 02:44:08,960 - src.snapshots - INFO - Starting choice of file type:
2024-01-12 02:44:08,961 - src.snapshots - INFO - FEniCS HDF5 file selected.
Loading Snapshots 2701: 100%|██████████| 2701/2701 [02:02<00:00, 22.12it/s]


In [8]:
# svd_params = {
#     "preprocessing_type": "min_max",
#     "trunc_basis": snapshots.shape[1],
#     "svd_type": "full_svd",  # ["full_svd", "randomized_svd"]
#     "power_iterations": 1,
#     "oversampling": 20,
# }

# svd_step = SVD(snapshots, svd_params)
# svd_step.fit()
# svd_step.s.shape

In [9]:
svd_params = {
    "trunc_basis": 10,
    "svd_type": "randomized_svd",  # ["full_svd", "randomized_svd"]
    "power_iterations": 1,
    "oversampling": 20,
}

snapshots_sample = snapshots[:500, :100]
svd_step_random = SVD(snapshots_sample, svd_params)
svd_step_random.fit()
svd_step_random.s.shape

b


(30,)

In [10]:
# import matplotlib.pyplot as plt

# plt.plot(svd_step_random.s[:220])
# plt.plot(svd_step.s[:220], "--")
# plt.show()

In [11]:
# dict_ae = {}

# ae_test = AutoEncoder(snapshots_sample, dict_ae)

In [12]:
# dict_ae = {
#     "batch_size": 10,
#     "num_epochs": 10,
#     "num_workers": 2,
#     "hidden_layer_size": 50,
#     "latent_size": 10,
# }

# ae_test_2 = AutoEncoderTrain(snapshots_sample, dict_ae)
# ae_test_2.fit()

In [30]:
dict_ae = {
    "batch_size": 10,
    "num_epochs": 10,
    "num_workers": 2,
    "number_of_hidden_layers": 4,
    "hidden_layers_sizes": [50, 25, 12, 6],
    "hidden_layers_activation_function": ["leaky_relu", "leaky_relu", "leaky_relu", ""],
    "hidden_layers_activation_function_parameters": [0.2, 0.2, 0.2, None],
    "decoder_activation_function": "sigmoid",
    "decoder_activation_function_parameter": None,
}

ae_test_2 = AutoEncoder(snapshots_sample, dict_ae)

In [31]:
ae_test_2.encoder

Sequential(
  (0): Linear(in_features=100, out_features=50, bias=True)
  (1): LeakyReLU(negative_slope=0.2)
  (2): Linear(in_features=50, out_features=25, bias=True)
  (3): LeakyReLU(negative_slope=0.2)
  (4): Linear(in_features=25, out_features=12, bias=True)
  (5): LeakyReLU(negative_slope=0.2)
  (6): Linear(in_features=12, out_features=6, bias=True)
)

In [32]:
ae_test_2.decoder

Sequential(
  (0): Linear(in_features=6, out_features=12, bias=True)
  (1): LeakyReLU(negative_slope=0.2)
  (2): Linear(in_features=12, out_features=25, bias=True)
  (3): LeakyReLU(negative_slope=0.2)
  (4): Linear(in_features=25, out_features=50, bias=True)
  (5): LeakyReLU(negative_slope=0.2)
  (6): Linear(in_features=50, out_features=100, bias=True)
  (7): Sigmoid()
)

In [33]:
ae_test_2 = AutoEncoderTrain(snapshots_sample, dict_ae)
ae_test_2.fit()

100%|██████████| 10/10 [00:04<00:00,  2.14it/s]
